In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import warnings
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
warnings.filterwarnings('ignore')

In [2]:
train_x = pd.read_csv('clean_train.csv')
test_x = pd.read_csv('clean_test.csv')

In [3]:
train_x.head()

,Unnamed: 0,id,loanAmnt,term,interestRate,installment,grade,employmentLength,annualIncome,issueDate,...,regionCode_48,regionCode_49,verificationStatus_1,verificationStatus_2,employmentTitle_cnts,employmentTitle_rank,title_cnts,title_rank,postCode_cnts,postCode_rank
0,0,0,35000.0,5,19.52,917.97,5,2.0,110000.0,2014-07-01,...,False,False,False,True,853,853,5201,5201,1618,1618
1,1,1,18000.0,5,18.49,461.90,4,5.0,46000.0,2012-08-01,...,False,False,False,True,79,79,25,25,2958,2958
2,2,2,12000.0,5,16.99,298.17,4,8.0,74000.0,2015-10-01,...,False,False,False,True,1,1,312481,312481,1325,1325
3,3,6,2050.0,3,7.69,63.95,1,9.0,35000.0,2014-10-01,...,False,False,False,False,1,1,312481,312480,636,636
4,4,7,11500.0,3,14.98,398.54,3,1.0,30000.0,2014-01-01,...,False,False,False,True,1,1,312481,312479,1615,1615


In [4]:
test_x.head()

,Unnamed: 0,id,loanAmnt,term,interestRate,installment,grade,employmentLength,annualIncome,issueDate,...,regionCode_48,regionCode_49,verificationStatus_1,verificationStatus_2,employmentTitle_cnts,employmentTitle_rank,title_cnts,title_rank,postCode_cnts,postCode_rank
0,0,800000,14000.0,3,10.99,458.28,2,10.0,80000.0,2014-07-01,...,False,False,False,False,8,8,78069,78069,161,161
1,1,800003,17500.0,5,14.31,410.02,3,4.0,37000.0,2014-11-01,...,False,False,True,False,6,6,30349,30349,274,274
2,2,800005,16000.0,3,5.32,481.84,1,10.0,90000.0,2017-05-01,...,False,False,False,False,325,325,78069,78068,718,718
3,3,800006,25000.0,5,14.99,594.62,3,10.0,126500.0,2016-11-01,...,False,False,False,True,329,329,78069,78067,317,317
4,4,800008,12000.0,5,16.55,295.34,4,3.0,58000.0,2015-10-01,...,False,False,True,False,308,308,78069,78066,529,529


In [5]:
features = [f for f in train_x.columns if f not in ['id', 'Unnamed: 0','isDefault', 'issueDate', 'policyCode']]

x_train = train_x[features]
x_test = test_x[features]

y_train = train_x['isDefault']

In [10]:
# from xgboost.sklearn import XGBClassifier
# clf = XGBClassifier(learning_rate= 0.05, n_estimators= 400, max_depth= 4, min_child_weight= 5, seed= 0,
#                     subsample= 0.7, colsample_bytree= 0.7, gamma= 0, reg_alpha= 5, reg_lambda=0.01,n_jobs = -1)
# clf.fit(x_train,y_train)
# clf.score(x_train,y_train)

0.8107686432462603

In [24]:
# 调用交叉验证函数
cross_val_xgboost(x_train, y_train)

************************************ 1 ************************************
Fold 1 - AUC Score: 0.7261
************************************ 2 ************************************
Fold 2 - AUC Score: 0.7278
************************************ 3 ************************************
Fold 3 - AUC Score: 0.7317
************************************ 4 ************************************
Fold 4 - AUC Score: 0.7296
************************************ 5 ************************************
Fold 5 - AUC Score: 0.7287
Mean AUC Score: 0.7287960764550485
Standard Deviation AUC Score: 0.0018658128518455291


In [25]:
#xgboost
from xgboost.sklearn import XGBClassifier

def cross_val_xgboost(x_train, y_train):
    folds = 5
    seed = 0
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

    train_auc_scores = []
    cv_auc_scores = []

    for i, (train_idx, valid_idx) in enumerate(kf.split(x_train)):
        print('************************************ {} ************************************'.format(str(i+1)))
        x_trn, y_trn = x_train.iloc[train_idx], y_train.iloc[train_idx]
        x_val, y_val = x_train.iloc[valid_idx], y_train.iloc[valid_idx]

        clf = XGBClassifier(
            learning_rate=0.05,
            n_estimators=400,
            max_depth=4,
            min_child_weight=5,
            seed=0,
            subsample=0.7,
            colsample_bytree=0.7,
            gamma=0,
            reg_alpha=5,
            reg_lambda=0.01,
            n_jobs=-1
        )

        clf.fit(x_trn, y_trn)
        train_probs = clf.predict_proba(x_trn)[:, 1]
        val_probs = clf.predict_proba(x_val)[:, 1]

        train_auc = roc_auc_score(y_trn, train_probs)
        cv_auc = roc_auc_score(y_val, val_probs)

        train_auc_scores.append(train_auc)
        cv_auc_scores.append(cv_auc)

        print("Fold {} - Train AUC Score: {:.4f}".format(i+1, train_auc))
        print("Fold {} - CV AUC Score: {:.4f}".format(i+1, cv_auc))

    mean_train_auc = np.mean(train_auc_scores)
    mean_cv_auc = np.mean(cv_auc_scores)

    print("Mean Train AUC Score:", mean_train_auc)
    print("Mean CV AUC Score:", mean_cv_auc)



In [26]:

 
cross_val_xgboost(x_train, y_train)

************************************ 1 ************************************
Fold 1 - Train AUC Score: 0.7403
Fold 1 - CV AUC Score: 0.7261
************************************ 2 ************************************
Fold 2 - Train AUC Score: 0.7401
Fold 2 - CV AUC Score: 0.7278
************************************ 3 ************************************
Fold 3 - Train AUC Score: 0.7393
Fold 3 - CV AUC Score: 0.7317
************************************ 4 ************************************
Fold 4 - Train AUC Score: 0.7392
Fold 4 - CV AUC Score: 0.7296
************************************ 5 ************************************
Fold 5 - Train AUC Score: 0.7399
Fold 5 - CV AUC Score: 0.7287
Mean Train AUC Score: 0.7397618500432301
Mean CV AUC Score: 0.7287960764550485


In [27]:
#cat
from catboost import CatBoostClassifier, Pool, cv


def cross_val_catboost(x_train, y_train):
    folds = 5
    seed = 0
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

    train_auc_scores = []
    cv_auc_scores = []

    for i, (train_idx, valid_idx) in enumerate(kf.split(x_train)):
        print('************************************ {} ************************************'.format(str(i+1)))
        x_trn, y_trn = x_train.iloc[train_idx], y_train.iloc[train_idx]
        x_val, y_val = x_train.iloc[valid_idx], y_train.iloc[valid_idx]

        clf = CatBoostClassifier(
            learning_rate=0.05,
            n_estimators=400,
            max_depth=4,
            l2_leaf_reg=5,
            subsample=0.7,
            colsample_bylevel=0.7,
            verbose=500,
            random_seed=0
        )

        clf.fit(x_trn, y_trn, eval_set=(x_val, y_val), early_stopping_rounds=200, verbose_eval=False)

        train_probs = clf.predict_proba(x_trn)[:, 1]
        val_probs = clf.predict_proba(x_val)[:, 1]

        train_auc = roc_auc_score(y_trn, train_probs)
        cv_auc = roc_auc_score(y_val, val_probs)

        train_auc_scores.append(train_auc)
        cv_auc_scores.append(cv_auc)

        print("Fold {} - Train AUC Score: {:.4f}".format(i+1, train_auc))
        print("Fold {} - CV AUC Score: {:.4f}".format(i+1, cv_auc))

    mean_train_auc = np.mean(train_auc_scores)
    mean_cv_auc = np.mean(cv_auc_scores)

    print("Mean Train AUC Score:", mean_train_auc)
    print("Mean CV AUC Score:", mean_cv_auc)




In [28]:

cross_val_catboost(x_train, y_train)

************************************ 1 ************************************
Fold 1 - Train AUC Score: 0.7308
Fold 1 - CV AUC Score: 0.7233
************************************ 2 ************************************
Fold 2 - Train AUC Score: 0.7305
Fold 2 - CV AUC Score: 0.7250
************************************ 3 ************************************
Fold 3 - Train AUC Score: 0.7298
Fold 3 - CV AUC Score: 0.7293
************************************ 4 ************************************
Fold 4 - Train AUC Score: 0.7299
Fold 4 - CV AUC Score: 0.7278
************************************ 5 ************************************
Fold 5 - Train AUC Score: 0.7300
Fold 5 - CV AUC Score: 0.7262
Mean Train AUC Score: 0.7301930622210622
Mean CV AUC Score: 0.7263190965987676
